In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib as mpl
mpl.rc('font', family='NanumGothic')
mpl.rc('axes', unicode_minus=False)

import warnings
warnings.filterwarnings("ignore")

In [3]:
PATH = '/content/drive/MyDrive/DACON/'
total = pd.read_csv(PATH+'한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')
test = pd.read_csv(PATH+'test.csv')
sub = pd.read_csv(PATH+'sample_submission.csv')

# 전처리

In [4]:
# 연월일 -> 시계열(연/월/일/요일) 변수 생성
total['연월일'] = pd.to_datetime(total['연월일'])
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday

In [5]:
def season(x):
  if x in [3, 4, 5]:
    return 0        # 0: 봄
  elif x in [6, 7, 8]:
    return 1        # 1: 여름
  elif x in [9, 10, 11]:
    return 2        # 2: 가을
  else:
    return 3        # 3: 겨울  

In [6]:
total['season'] = total['month'].apply(season)
total['workday'] = total['weekday'].apply(lambda x: 1 if x in [0,1,2,3,4] else 0)

In [7]:
d_map = {}

for i, d in enumerate(total['구분'].unique()):
    d_map[d] = i
total['구분'] = total['구분'].map(d_map)

In [8]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

In [9]:
features = ['시간', '구분', 'year', 'month', 'day', 'weekday', 'season', 'workday']

train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

# 회귀분석

In [11]:
from statsmodels.formula.api import ols

In [18]:
model = ols('공급량 ~ 시간+구분+month+day+weekday', data=train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    공급량   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     2397.
Date:                Sun, 31 Oct 2021   Prob (F-statistic):               0.00
Time:                        11:56:22   Log-Likelihood:            -2.5193e+06
No. Observations:              306768   AIC:                         5.039e+06
Df Residuals:                  306762   BIC:                         5.039e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1017.4936      6.335    160.627      0.000    1005.078    1029.909
시간            13.2365      0.233     56.892      0.000      12.781      13.693
구분            23.8199      0.805     29.580      0.000      22.242      25.398
month        -39.9815      0.467    -85.606      0.000     -40.897     -39.066
day           -0.1376      0.183     -0.752      0.452      -0.496       0.221
weekday      -18.8432      0.806    -23.392      0.000     -20.422     -17.264
==============================================================================
Omnibus:                    70230.741   Durbin-Watson:                   0.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           141520.308
Skew:                           1.379   Prob(JB):                         0.00
Kurtosis:                       4.863   Cond. No.                         90.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* 일단 직접 만든 변수(season, workday) 제외하고 기본 변수로만 회귀돌림
* day 빼고는 다 유의하게 나왔고 모형도 유의함  
* 근데 결정계수 0.038???????이라는 처참한 값을 얻음 

In [17]:
model = ols('공급량 ~ 시간+구분+month+weekday', data=train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    공급량   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     2996.
Date:                Sun, 31 Oct 2021   Prob (F-statistic):               0.00
Time:                        11:56:15   Log-Likelihood:            -2.5193e+06
No. Observations:              306768   AIC:                         5.039e+06
Df Residuals:                  306763   BIC:                         5.039e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1015.3560      5.660    179.386      0.000    1004.262    1026.450
시간            13.2365      0.233     56.892      0.000      12.781      13.693
구분            23.8199      0.805     29.580      0.000      22.242      25.398
month        -39.9853      0.467    -85.619      0.000     -40.901     -39.070
weekday      -18.8437      0.806    -23.393      0.000     -20.423     -17.265
==============================================================================
Omnibus:                    70225.761   Durbin-Watson:                   0.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           141504.068
Skew:                           1.378   Prob(JB):                         0.00
Kurtosis:                       4.863   Cond. No.                         56.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* day를 제거해도 모형 설명력이 좋아지진 않음

In [16]:
model = ols('공급량 ~ 시간+month', data=train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    공급량   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     5256.
Date:                Sun, 31 Oct 2021   Prob (F-statistic):               0.00
Time:                        11:55:01   Log-Likelihood:            -2.5200e+06
No. Observations:              306768   AIC:                         5.040e+06
Df Residuals:                  306765   BIC:                         5.040e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1030.1483      4.520    227.924      0.000    1021.290    1039.007
시간            13.2365      0.233     56.761      0.000      12.779      13.694
month        -39.9692      0.468    -85.387      0.000     -40.887     -39.052
==============================================================================
Omnibus:                    74699.338   Durbin-Watson:                   0.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           158403.705
Skew:                           1.434   Prob(JB):                         0.00
Kurtosis:                       5.042   Cond. No.                         43.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* 베이스라인처럼 시간이랑 month만 두고 돌렸는데 더 쓰레기가 됨

In [22]:
model = ols('공급량 ~ 시간+구분+season+workday', data=train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    공급량   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                 1.192e+04
Date:                Sun, 31 Oct 2021   Prob (F-statistic):               0.00
Time:                        11:57:27   Log-Likelihood:            -2.5030e+06
No. Observations:              306768   AIC:                         5.006e+06
Df Residuals:                  306763   BIC:                         5.006e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    204.3897      5.014     40.760      0.000     194.562     214.218
시간            13.2365      0.221     59.994      0.000      12.804      13.669
구분            23.8199      0.764     31.193      0.000      22.323      25.317
season       280.5899      1.367    205.221      0.000     277.910     283.270
workday      105.2074      3.380     31.125      0.000      98.582     111.832
==============================================================================
Omnibus:                    49271.849   Durbin-Watson:                   0.027
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            81464.451
Skew:                           1.085   Prob(JB):                         0.00
Kurtosis:                       4.292   Cond. No.                         51.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* 그나마 높아지긴 했지만... 사실 stepwise 코드를 못 찾아서 제대로 모형 적합 못함
* 아무튼 오늘은 시간+구분+season+workday 네 개 변수 가지고 해보기로 함

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [25]:
features = ['시간', '구분', 'season', 'workday']

train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

lr = LinearRegression()
lr.fit(train_x, train_y)
y_preds = lr.predict(val_x)
mse = mean_squared_error(val_y, y_preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(val_y, y_preds)

print('MSE: {0:.3f}, RMSE: {1:.3f}, MAE: {1:.3f}'.format(mse, rmse, mae))
print('Varinace score: {0:.3f}'.format(r2_score(val_y, y_preds)))

MSE: 875308.580, RMSE: 935.579, MAE: 935.579
Varinace score: 0.141


* 왜 이렇게 나오지? 정말 곤란해ㅜ

In [10]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import validation_curve, train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

features = ['시간', '구분', 'season', 'workday']
y = total['공급량']
#X = total.drop(['공급량', 'year', '연월일'], axis=1, inplace=False)
X = total[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
std = StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

In [32]:
# # Ridge + GridSearchCV
# ridge = Ridge()
# params = {'alpha': np.linspace(0.01, 1.0, 100)} # lambda의 역할과 동일
# ridge_cv = GridSearchCV(ridge, params, scoring='r2', cv=5, n_jobs=-1)
# ridge_cv.fit(X_train_std, y_train)
# opt_r = ridge_cv.best_estimator_
# r2_ridge = r2_score(y_test, opt_r.predict(X_test_std))

# # Lasso + GridSearchCV
# lasso = Lasso()
# lasso_cv = GridSearchCV(lasso, params, scoring='r2', cv=5, n_jobs=-1)
# lasso_cv.fit(X_train_std, y_train)
# opt_l = lasso_cv.best_estimator_
# r2_lasso = r2_score(y_test, opt_l.predict(X_test_std))

# # Elasticnet + GridSearchCV
# elastic = ElasticNet()
# elastic_cv = GridSearchCV(elastic, params, scoring='r2', cv=5, n_jobs=-1)
# elastic_cv.fit(X_train_std, y_train)
# opt_e = elastic_cv.best_estimator_
# r2_elastic = r2_score(y_test, opt_e.predict(X_test_std))

# print('Ridge R^2 score: {0:.4f}, Lasso R^2 score: {1:.4f}, Elastic R^2 socre: {2:.4f}'.format(r2_ridge,r2_lasso,r2_elastic))

Ridge R^2 score: 0.1894, Lasso R^2 score: 0.1894, Elastic R^2 socre: 0.1894


* 뭔가 잘못한건지 아님 원래 회귀로 돌리면 이따군지 잘 모르겠음^^;;

# 학습/예측/제출

In [12]:
model = Lasso()
params = {'alpha': np.linspace(0.01, 1.0, 100)} # lambda의 역할과 동일
model_cv = GridSearchCV(model, params, scoring='r2', cv=5, n_jobs=-1)
model_cv.fit(X_train_std, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.1...
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2

In [15]:
test

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A
3,2019-01-01 04 A
4,2019-01-01 05 A
...,...
15115,2019-03-31 20 H
15116,2019-03-31 21 H
15117,2019-03-31 22 H
15118,2019-03-31 23 H


In [16]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [17]:
test['구분'] = test['구분'].map(d_map)
test['일자'] = pd.to_datetime(test['일자'])
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday
test['workday'] = test['weekday'].apply(lambda x: 1 if x in [0,1,2,3,4] else 0)
test['season'] = test['month'].apply(season)

In [19]:
test_x = test[features]
preds = model_cv.predict(test_x)

In [20]:
sub['공급량'] = preds
sub.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,2056.480110
1,2019-01-01 02 A,2148.955668
2,2019-01-01 03 A,2241.431226
3,2019-01-01 04 A,2333.906783
4,2019-01-01 05 A,2426.382341


In [21]:
sub.to_csv('jk_1031_lasso.csv', index=False)